## Aanmaken van de database

In de volgende opdrachten voer je allerlei queries uit op een database.
Je moeten dan eerst wel een database met inhoud hebben.
Met de onderstaande opdrachten maak je deze database.
Deze opdrachten hoef je maar één keer uit te voeren: de database blijft bestaan, met je veranderingen.
Je kunt deze opdrachten ook uitvoeren om opnieuw te beginnen, in een goed gedefinieerde toestand.

In [ ]:
import sqlite3
db = sqlite3.connect('example.db')
cursor = db.cursor()

We maken de tabel(len) aan.
We verwijderen eerst een eventueel bestaande versie van de tabel(len):
we hebben dan een goed gedefinieerde toestand.

> Opmerking: er zijn kleine verschillen in de notatie van de constraints bij het aanmaken van een tabel; MySQL gebruikt bijvoorbeeld een andere notatie dan Oracle.

In [ ]:
cursor.execute('''DROP TABLE IF EXISTS leden;''')
cursor.execute('''CREATE TABLE leden(
                    lidnr INTEGER PRIMARY KEY,
                    voornaam VARCHAR(255) NOT NULL,
                    achternaam VARCHAR(255) NOT NULL,
                    email VARCHAR(255) NOT NULL UNIQUE
                  );''')

We hebben een voorbeeld-inhoud van de tabel(len) in csv-bestanden.
(Een voordeel daarvan is dat we deze gemakkelijk kunnen aanpassen in een teksteditor.)
Voor het importeren van een csv-bestand gebruiken we een speciale SQLite-opdracht, via de shell.
(Een alternatief is om dit bestand te importeren via pandas.)

In [ ]:
%%bash
sqlite3 example.db
.mode csv
.import leden.csv leden

Hieronder een voorbeeld van een SQL-opdracht die we rechtstreeks in SQLite uitvoeren.

In [ ]:
%%bash
sqlite3 example.db
SELECT * FROM leden;

Eenzelfde opdracht, nu als onderdeel van een Python-programma:

In [ ]:
cursor.execute('''SELECT * FROM leden;''')
for row in cursor:
    print(row)

We volgende hetzelfde patroon voor de tweede tabel: inschrijvingen.

In [ ]:
cursor.execute('''DROP TABLE IF EXISTS inschrijvingen;''')
cursor.execute('''CREATE TABLE inschrijvingen(
                  inschrijfnr INTEGER, 
                  lidnr INTEGER,
                  datum VARCHAR(255) NOT NULL,
                  maaltijd VARCHAR(255),
                  PRIMARY KEY (inschrijfnr),
                  FOREIGN KEY (lidnr) REFERENCES leden (lidnr),
                  CONSTRAINT name UNIQUE (lidnr, datum)
                  );''')

In [ ]:
%%bash
sqlite3 example.db
.mode csv
.import inschrijvingen.csv inschrijvingen

In [ ]:
cursor.execute('''SELECT * FROM inschrijvingen;''')
for row in cursor:
    print(row)

## Database connection en cursor

De eerste stap in het gebruik van database-opdrachten in Python is het maken van een verbinding (connection) met de database.
Met behulp van deze connection maak je een cursor-object, waarmee je de database-queries kunt uitvoeren.

Bij veranderingen in de database is het belangrijk om `db.commit()` uit te voeren: de veranderingen worden daarmee permanent gemaakt.

In [ ]:
import sqlite3
db = sqlite3.connect('example.db')
cursor = db.cursor()

In [ ]:
cursor.execute('''SELECT * FROM leden;''')
for row in cursor:
    print(row)

## Selectie

Je kunt de waarden van een tabel selecteren op basis van een voorwaarde.
In de SQL SELECT-opdracht geef je de conditie op na `WHERE`.
Dit kunt condities samenstellen met behulp van logische en rekenkundige opereratoren, zoals vergelijkingen.

In [ ]:
cursor.execute('''SELECT * FROM leden lid
                  WHERE lid.lidnr > 1;''')
for row in cursor:
    print(row)

## Projectie

Je kunt selectie combineren met *projectie*: daarin geef je aan welke kolommen in het resultaat voorkomen.
De andere kolommen worden weggelaten.

In de SQL SELECT-opdracht geef je de namen van de kolommen van de projectie als eerste op.
De volgorde mag daarbij anders zijn dan in de oorspronkelijke tabel:
je kijgt als resultaat een tabel met de kolommen die je opgeeft.

In [ ]:
cursor.execute('''SELECT email, achternaam FROM leden;''')
for row in cursor:
    print(row)

Een tabel kan ook uit een enkele kolom bestaan:

In [ ]:
cursor.execute('''SELECT voornaam FROM leden;''')
for row in cursor:
    print(row)

(Zoals je ziet komen hierin rijen dubbel voor. Als je geen dubbele rijen wilt, gebruik je `SELECT UNIQUE`. - ????)

## Cartesisch product

Het cartesisch product van twee relaties (tabellen) bevat *alle combinaties* van de rijen van beide tabellen.

* vraag: hoeveel rijen heeft een cartesisch product van twee tabellen met elk 1000 rijen? (1000; 2000; 1.000.000?)

In [ ]:
cursor.execute('''SELECT *
                  FROM leden, inschrijvingen;''')
for row in cursor:
    print(row)

De meeste van deze combinaties zijn zinloos:
in het onderstaande voorbeeld zijn we alleen geïnteresseerd in de rijen waarvan de lidnr's gelijk zijn.
We selecteren de relevante rijen van het cartesisch product door middel van een `WHERE`-voorwaarde.
Dit is een normaal patroon voor een *join*.

In [ ]:
cursor.execute('''SELECT *
                  FROM leden lid, inschrijvingen ins
                  WHERE lid.lidnr=ins.lidnr;''')
for row in cursor:
    print(row)

We kunnen bovenstaande combinatie (join) van tabellen combineren met andere selectie-voorwaarden, en met project, zoals hieronder:


In [ ]:
cursor.execute('''SELECT lid.voornaam, lid.achternaam, lid.email, 
                         ins.maaltijd
                  FROM leden lid, inschrijvingen ins
                  WHERE lid.lidnr=ins.lidnr AND ins.datum='23-06-2019'; ''')
for row in cursor:
    print(row)

## Normalisatie

De tabellen die we hierboven gebruiken zijn in normaalvorm: deze bevatten geen redundante gegevens.
Elk basisgegeven komt maar één keer voor.
Dat maakt het eenvoudiger om de database te veranderen op een consistende manier.

De volgende tabel (als resultaat van een "join" bewerking) is niet genormaliseerd:
je ziet dat dezelfde voornaam, achternaam en mail-adres meerdere malen voorkomen.
Dat is de reden waarom we deze tabel berekenen, en de basisgegevens in verschillende tabellen ondergebracht hebben.


In [ ]:
cursor.execute('''SELECT *
                  FROM leden lid, inschrijvingen ins
                  WHERE lid.lidnr=ins.lidnr;''')
for row in cursor:
    print(row)

## Schema

Het *schema* van een database beschrijft de structuur: de tabellen, kolomnamen, types en constraints.
Dit kun je bijvoorbeeld weergeven in de vorm van de "create table" opdrachten:

In [ ]:
%%bash
sqlite3 example.db
.databases
.schema

Je kunt het schema (de structuur) van een tabel ook in SQL opvragen.

In [ ]:
cursor.execute('''SELECT sql FROM sqlite_master WHERE name='leden';''')
for row in cursor:
    print(row[0])

Voor de kolomnamen van sqlite_master, zie: https://www.techonthenet.com/sqlite/sys_tables/index.php

* type
* name
* tbl_name
* rootpage
* sql

**Opmerking** Eén van de eigenschappen van een relationeel database management systeem (RDBMS) is dat het schema van een database zelf in een (beheer)database opgeslagen is. Via deze database kun je dan de namen van de tabellen, de namen en types van de kolommen, en de constraints opvragen.

## Opmerkingen

* zie: http://www.sqlitetutorial.net/sqlite-import-csv/
* het is handiger om de tabel eerst aan te maken, en dan de CSV te importeren:
  dan kun je het type van de kolommen aangeven, de csv bevat alleen de kolomnamen
* hoe geven we aan wat de key is?
* naamgeving: gebruiken we voor entiteiten en voor de bijbehorende tabellen enkelvoud of meervoud? (Volgens mij is de meer gebruikelijke methode: enkelvoud.)
    * ==> als in Toon's boek: (i) voor Entiteiten, enkelvoud; (ii) voor tabellen, meervoud; (iii) bij condities (`WHERE`), als enkelvoud logischer is, alias in enkelvoud gebruiken.

In [ ]:
cursor.execute('''INSERT INTO leden (voornaam, achternaam, email) 
                  VALUES (?, ?, ?);''', 
                  ('Jan', 'Visser', 'jvisser@gmail.com'))
db.commit()

In [ ]:
cursor.execute('''SELECT * FROM leden''')
for row in cursor:
    print(row)

In [ ]:
cursor.execute('''INSERT INTO leden (voornaam, achternaam, email) 
                  VALUES (?, ?, ?);''', 
                  ('Jan', 'Visser', 'jvisser@gmail.com'))

In [ ]:
cursor.execute('''SELECT * FROM leden;''')
for row in cursor:
    print(row)

In [ ]:
cursor.execute('''INSERT INTO leden (lidnr, voornaam, achternaam, email) 
                  VALUES (?, ?, ?, ?);''', 
                  (4, 'Jan', 'Visser', 'jvisser@gmail.com'))

**Opmerking**: SQLite heeft voor een INTEGER KEY automatisch een AUTOINCREMENT; het gebruik van het keyword AUTOINCREMENT geeft dan een andere betekenis, en moet eigenlijk vermeden worden. (https://www.sqlite.org/autoinc.html)
Dit kan bij andere database-systemen anders zijn!

In [ ]:
try:
    cursor.execute('''INSERT INTO leden (voornaam, achternaam, email) 
                  VALUES (?, ?, ?);''', 
                  ('Jan', 'Visser', 'jvisser@gmail.com'))
except sqlite3.IntegrityError as e:
    print(e)
    

Met behulp van het Python exception-mechamisme kun je fouten in de uitvoering van een sqlite-opdracht opvangen, en omzetten in een foutmeling (aan de gebruiker?)

(We kunnen eventueel verschillende soorten fouten onderscheiden, bijv. integriteitsfouten zoals hierboven.)

Welk soort fouten verwachten we?

* toevoegen van een gebruiker die al bestaat (bestaande email, zoals hierboven);
* toevoegen van een inschrijving die al bestaat (combinatie van lidnr en datum???); kun je dit als constraint invoeren? Ja! Dat betekent dat we deze niet zelf hoeven te controleren: in het algemeen is het beter om de DB het werk te laten doen. Dit vereenvoudigt onze programma's (maar we moeten wel de exceptions uitprogrammeren...)

**Vraag:** hoe kun je een list omzetten in een reeks waarden, bijvoorbeeld voor `format`?
Dat kan met behulp van de splat-operatie: `*list`.

## CRUD

Met het acroniem CRUD geven we de basisoperaties op een database aan: Create, Read, Update, en Delete.
Hieronder geven we de SQL-opdrachten die daarmee overeenkomen. We geven daarbij ook aan hoe deze samenhangen met eventuele constraints van de database.

De CRD-operaties veranderen de database: je maakt deze veranderingen permanent met behulp van `db.commit()`. Vergeet deze niet in je programma's! Je kunt meerdere CRD-operaties combineren voor deze commit: je krijgt dan een *transactie* die uit meerdere opdrachten bestaat.

### Create (INSERT)

Met de INSERT opdracht voegen we een nieuwe rij aan een tabel toe (create):

In [ ]:
inschrijving = (3, '30-06-2019', 'Maaltijd A')
cursor.execute('''INSERT INTO inschrijvingen (lidnr, datum, maaltijd) 
                  VALUES (?, ?, ?);''', 
                  inschrijving)

In [ ]:
cursor.execute('''SELECT * FROM inschrijvingen''')
for row in cursor:
    print(row)

Nogmaals uitvoeren geeft een IntegrityError (exception):
we hebben immers aangegeven dat de combinatie van `lidnr` en `datum` uniek moet zijn (constraint).

In [ ]:
cursor.execute('''INSERT INTO inschrijvingen (lidnr, datum, maaltijd) 
                  VALUES (?, ?, ?);''', 
                  inschrijving)

Met behulp van het Python exception-mechamisme kunnen we dergelijke fouten opvangen,
en eventueel omzetten in een andere foutmelding - bijvoorbeeld een foutmelding naar de gebruiker,
dat hij zich al eerder voor deze event ingeschreven heebt.

In [ ]:
try:
    cursor.execute('''INSERT INTO inschrijvingen (lidnr, datum, maaltijd) 
                  VALUES (?, ?, ?);''', 
                  inschrijving)
except sqlite3.IntegrityError as e:
    print(e)
    print('Gebruiker {} heeft zich eerder aangemeld voor {}'.format(*inschrijving))

**Opmerking** de splat-operatie `*inschrijving` zet de lijst `inschrijving` om in een lijst van parameters - hier voor `format`.

### Read (SELECT)

Met de SELECT-opdracht zoals we hierboven gebruikt hebben kun je de data uit de database lezen.
Het resultaat van SELECT is altijd een tabel; in sommige gevallen bestaat deze uit 1 rij, als we een bestaande rij willen lezen. Soms is de tabel leeg: er zijn dan geen rijen gevonden die aan de voorwaarde voldoen.

## Update (UPDATE)

Met de UPDATE opdracht kun je een bestaande rij selectief veranderen.
In het onderstaande voorbeeld passen we de maaltijd-waarde aan:

In [ ]:
maaltijd = 'maaltijd B'
cursor.execute('''UPDATE inschrijvingen
                  SET maaltijd= ?
                  WHERE lidnr=3 AND datum='30-06-2019';''',
                  (maaltijd,))

In [ ]:
cursor.execute('''SELECT * FROM inschrijvingen;''')
for row in cursor:
    print(row)

* Ga na dat herhalen van deze opdracht hetzelfde resultaat oplevert.
* Ga na dat een update voor een lege selectie geen foutmelding oplevert.
* Ga na wat er gebeurt als je de datum-voorwaarde weglaat. (Kun je in een UPDATE opdracht meerdere rijen kunt veranderen?)

### Delete (DELETE)

Met de DELETE opdracht verwijder je een rij uit de database.

In [ ]:
selection = (3, '30-06-2019')
cursor.execute('''DELETE FROM inschrijvingen
                  WHERE lidnr=? and datum=?;''',
                  selection)

In [ ]:
cursor.execute('''SELECT * FROM inschrijvingen;''')
for row in cursor:
    print(row)

Herhalen van deze opdracht levert geen problemen op: het verwijderen van een niet-bestaand element slaagt "per definitie". DELETE is een *idempotente* opdracht: deze heeft dezelfde betekenis als je deze eenmaal of vaker uitvoert.

## Opmerkingen/ToDo's

ToDo:

* [ ] toevoegen vb. van fetchone met tuple-assignment